In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
import pandas as pd
import torch
import transformers
import evaluate

# Загрузка датасета




In [4]:
df = pd.read_csv('drive/MyDrive/IMDB Dataset.csv')
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

# Подготовка данных

In [7]:
from transformers import AutoTokenizer
from torch.utils.data import Dataset, DataLoader

# Разделение на тренировочный, тестовый и валидационный наборы данных
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.25, random_state=42)

# Инициализация токенизатора
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")

# Создание кастомного датасета
class DataProblemsDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=128):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.texts = dataframe["review"].tolist()
        self.labels = dataframe["sentiment"].astype('category').cat.codes.tolist()  # Категоризируем классы
        print(f"всего классов: {len(set(self.labels))}")

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(text, padding='max_length', truncation=True, max_length=self.max_length, return_tensors="pt")
        item = {key: val.squeeze(0) for key, val in encoding.items()}
        item["labels"] = torch.tensor(label, dtype=torch.long)
        return item

# Подготовка DataLoader для обучения
train_dataset = DataProblemsDataset(train_df, tokenizer)
val_dataset = DataProblemsDataset(val_df, tokenizer)
test_dataset = DataProblemsDataset(test_df, tokenizer)

# Параметры DataLoader
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# Проверка размеров наборов данных
print("Размер тренировочного набора:", len(train_df))
print("Размер валидационного набора:", len(val_df))
print("Размер тестового набора:", len(test_df))

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

всего классов: 2
всего классов: 2
всего классов: 2
Размер тренировочного набора: 30000
Размер валидационного набора: 10000
Размер тестового набора: 10000


In [8]:
train_df.head()

,review,sentiment
29605,This is one of those games where you love it t...,positive
33044,...this film noire set piece suffers from murk...,positive
16594,I sat through this turkey because I hadn't see...,negative
21719,An excellent film depicting the cross currents...,positive
24972,What a cast of actors and actresses in this Co...,positive


Класс для использования backbone

In [9]:
import torch
import torch.nn as nn
from transformers import AutoModel, AutoConfig
from typing import Union

class TransformerClassificationModel(nn.Module):
    def __init__(self, base_transformer_model: Union[str, nn.Module], num_classes: int = 7):
        super(TransformerClassificationModel, self).__init__()

        # Загружаем конфигурацию и модель (backbone) из Hugging Face
        if isinstance(base_transformer_model, str):
            self.config = AutoConfig.from_pretrained(base_transformer_model)
            self.backbone = AutoModel.from_pretrained(base_transformer_model, config=self.config)
        elif isinstance(base_transformer_model, nn.Module):
            self.backbone = base_transformer_model
            self.config = self.backbone.config
        else:
            raise ValueError("base_transformer_model должен быть строкой (название модели) или объектом nn.Module")

        # Дополнительный классификационный слой
        # Используем выходной размер скрытого слоя трансформера для настройки линейного слоя
        self.classifier = nn.Linear(self.config.hidden_size, num_classes)

    def forward(self, inputs):
        # Пропускаем данные через backbone
        backbone_outputs = self.backbone(**inputs)

        # Обычно используется [CLS] токен для классификации, который находится в нулевой позиции
        # Возьмем скрытые состояния первого токена ([CLS]) из последнего слоя
        cls_output = backbone_outputs.last_hidden_state[:, 0, :]

        # Пропускаем через классификационный слой
        logits = self.classifier(cls_output)

        # Возвращаем словарь с логитами
        outputs = {"logits": logits}
        return outputs

Функция для заморозки backbone

In [10]:
def freeze_backbone_function(model: TransformerClassificationModel):
    for param in model.backbone.parameters():
        param.requires_grad = False

    return model

Дообучение трансформера

In [11]:
import copy
import torch
from torch.optim import Adam
from tqdm import tqdm

def train_transformer(transformer_model, train_loader, val_loader=None, freeze_backbone=True,
                      num_epochs=3, learning_rate=1e-5, device="cuda" if torch.cuda.is_available() else "cpu"):
    # Копируем модель, чтобы не изменять оригинал
    model = copy.deepcopy(transformer_model).to(device)

    # Замораживаем backbone, если это указано
    if freeze_backbone:
        model = freeze_backbone_function(model)

    # Определяем оптимизатор
    optimizer = Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=learning_rate)

    # Режим обучения
    model.train()

    for epoch in range(num_epochs):
        total_loss = 0
        # Прогресс-бар для удобного отслеживания обучения
        progress_bar = tqdm(train_loader, desc=f"Эпоха {epoch + 1}/{num_epochs}")

        for batch in progress_bar:
            # Подготавливаем данные для передачи на устройство
            inputs = {key: val.to(device) for key, val in batch.items() if key != 'labels'}
            labels = batch['labels'].to(device)

            # Обнуляем градиенты
            optimizer.zero_grad()

            # Прямое распространение и расчет потерь
            outputs = model(inputs)
            logits = outputs["logits"]
            loss = torch.nn.functional.cross_entropy(logits, labels)

            # Обратное распространение и обновление весов
            loss.backward()
            optimizer.step()

            # Накопление общей потери для отслеживания
            total_loss += loss.item()
            progress_bar.set_postfix(loss=loss.item())

        # Средняя потеря за эпоху
        avg_loss = total_loss / len(train_loader)
        print(f"Средняя потеря за эпоху {epoch + 1}: {avg_loss:.4f}")

        # Оценка на валидационном наборе, если он предоставлен
        if val_loader:
            model.eval()
            val_loss = 0
            correct_predictions = 0
            total_predictions = 0

            with torch.no_grad():
                for val_batch in val_loader:
                    inputs = {key: val.to(device) for key, val in val_batch.items() if key != 'labels'}
                    labels = val_batch['labels'].to(device)
                    outputs = model(inputs)
                    logits = outputs["logits"]

                    val_loss += torch.nn.functional.cross_entropy(logits, labels).item()

                    # Предсказания и подсчет точных
                    preds = torch.argmax(logits, dim=1)
                    correct_predictions += (preds == labels).sum().item()
                    total_predictions += labels.size(0)

            avg_val_loss = val_loss / len(val_loader)
            accuracy = correct_predictions / total_predictions
            print(f"Валидационная потеря: {avg_val_loss:.4f}, Точность: {accuracy:.4f}")

            # Возвращаемся в режим обучения
            model.train()

    # Возвращаем дообученную модель
    finetuned_model = model
    return finetuned_model

Файн-тюнинг bert-base-uncased

In [13]:
# Дообучение с замороженным backbone
rubert_tiny_transformer_model = TransformerClassificationModel("google-bert/bert-base-uncased", num_classes=2)
rubert_tiny_finetuned_with_freezed_backbone = train_transformer(
    rubert_tiny_transformer_model, train_loader, val_loader, freeze_backbone=True, num_epochs=10
)

# Дообучение с размороженным backbone
rubert_tiny_transformer_model = TransformerClassificationModel("google-bert/bert-base-uncased", num_classes=2)
rubert_tiny_full_finetuned = train_transformer(
    rubert_tiny_transformer_model, train_loader, val_loader, freeze_backbone=False, num_epochs=10
)

Эпоха 1/10: 100%|██████████| 1875/1875 [04:05<00:00,  7.63it/s, loss=0.616]


Средняя потеря за эпоху 1: 0.6604
Валидационная потеря: 0.6292, Точность: 0.6968


Эпоха 2/10: 100%|██████████| 1875/1875 [04:06<00:00,  7.59it/s, loss=0.669]


Средняя потеря за эпоху 2: 0.6239
Валидационная потеря: 0.5940, Точность: 0.7353


Эпоха 3/10: 100%|██████████| 1875/1875 [04:06<00:00,  7.62it/s, loss=0.574]


...

Средняя потеря за эпоху 10: 0.5117
Валидационная потеря: 0.4908, Точность: 0.7749


Эпоха 1/10: 100%|██████████| 1875/1875 [11:54<00:00,  2.62it/s, loss=0.182]


Средняя потеря за эпоху 1: 0.3186
Валидационная потеря: 0.2679, Точность: 0.8825


Эпоха 2/10:   3%|▎         | 61/1875 [00:23<11:43,  2.58it/s, loss=0.144]


KeyboardInterrupt: 

In [14]:
# Дообучение с размороженным backbone
rubert_transformer_model = TransformerClassificationModel("google-bert/bert-base-uncased", num_classes=2)
rubert_full_finetuned = train_transformer(
    rubert_tiny_transformer_model, train_loader, val_loader, freeze_backbone=False, num_epochs=3
)

Эпоха 1/3: 100%|██████████| 1875/1875 [11:53<00:00,  2.63it/s, loss=0.244]


Средняя потеря за эпоху 1: 0.3149
Валидационная потеря: 0.2702, Точность: 0.8852


Эпоха 2/3: 100%|██████████| 1875/1875 [11:53<00:00,  2.63it/s, loss=0.067]


Средняя потеря за эпоху 2: 0.1992
Валидационная потеря: 0.3031, Точность: 0.8839


Эпоха 3/3: 100%|██████████| 1875/1875 [11:54<00:00,  2.63it/s, loss=0.0375]


Средняя потеря за эпоху 3: 0.1162
Валидационная потеря: 0.3261, Точность: 0.8883


Сохранение модели

In [26]:
torch.save(rubert_full_finetuned, 'drive/MyDrive/finetuned.pth')

Инференс

In [43]:
# Создание пустого датафрейма df2
df2 = pd.DataFrame(columns=['review', 'sentiment'])

# Копирование строки из df в df2 с помощью метода .loc
df2.loc[29605] = df.loc[29605]
df2.loc[33044] = df.loc[33044]
df2.loc[16594] = df.loc[16594]
df2.loc[21719] = df.loc[21719]
df2.loc[24792] = df.loc[24792]
df2

,review,sentiment
29605,This is one of those games where you love it t...,positive
33044,...this film noire set piece suffers from murk...,positive
16594,I sat through this turkey because I hadn't see...,negative
21719,An excellent film depicting the cross currents...,positive
24792,i will like to order this movie for the women ...,positive


In [45]:
impl_dataset = DataProblemsDataset(df2, tokenizer)
impl_loader = DataLoader(impl_dataset, batch_size=batch_size)

всего классов: 2


In [51]:
progress_bar = tqdm(impl_loader)

for batch in progress_bar:
  # Подготавливаем данные для передачи на устройство
  inputs = {key: val.to(device) for key, val in batch.items() if key != 'labels'}
  labels = batch['labels'].to(device)

  # Прямое распространение и расчет потерь
  with torch.no_grad():
    outputs = rubert_full_finetuned(inputs)
    logits = outputs["logits"]
    pred_class = torch.argmax(logits, dim=1)

pred_class.tolist()


100%|██████████| 1/1 [00:00<00:00, 69.05it/s]


[1, 1, 0, 1, 1]

In [59]:
for i in range(len(df2)):
  print(f"Текст: {df2['review'].values[i]}, \nпредсказанный класс: {'позитивный' if pred_class[i] == 1 else 'негативный'}\n")

Текст: This is one of those games where you love it to bits or hate it to shreds.Even being a hardcore Mario fan can make you dislike this game.You can hate it because it is 2 short and somewhat boring and easy.Or you can love it because it is a mixture of amazing graphics(not a Nintendo fan huge excitement) music or game play.I know a lot of people that say it is amazing,and others who think its the worst Mario ever.It really depends on the type of Nintendo fan you are.I personally love this game and I think it is the best wii game,but you should determine that for yourself.So I think you should absolutely get it if you are the right Nintendo fan.But If you love The classics too much,you may not like it.So try it out yourself., 
предсказанный класс: позитивный

Текст: ...this film noire set piece suffers from murky sound (at least, as shown on the inadequate equipment of both the Seattle and Maine film festivals) and murkier plotting, while Rickman suffers from an American accent, old